# Multi-Agent Financial Analysis System
Converted automatically to Jupyter Notebook.

In [10]:
""" 
Multi-Agent Financial Analysis System (Rewritten)
- Ensures agents only use provided knowledge bases (text files & CSVs)
- Forces retrieval, builds KBs, and prevents hallucinations by passing retrieved
  content as strict context to agents.

Instructions:
1. Place your .txt files in `financeAgent/data/` and .csv files in `csvAgent/data/`.
2. Run this notebook/script. It will load, build, and index the KBs.
3. Use the `ask_team(query)` helper which retrieves relevant documents first,
   then calls the agents with ONLY that retrieved context.

Make changes easily: search, chunk, or retrieval parameters are at the top as constants.
"""

# Standard library
import os
import glob
import pathlib
import json
import traceback
from typing import List, Dict, Any

# Third-party
import pandas as pd

# ====== AGNO imports ======
from agno.knowledge.text import TextKnowledgeBase
from agno.knowledge.csv import CSVKnowledgeBase
from agno.agent import Agent
from agno.models.google import Gemini

# ---------------------- CONFIG ----------------------
FINANCE_DIR = pathlib.Path("financeAgent/data")
CSV_DIR = pathlib.Path("csvAgent/data")

TOP_K_TEXT = 6
CSV_TOP_K = 100

FINANCE_DIR.mkdir(parents=True, exist_ok=True)
CSV_DIR.mkdir(parents=True, exist_ok=True)

def safe_read_text(file_path: str) -> str:
    encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
    for enc in encodings:
        try:
            with open(file_path, 'r', encoding=enc) as f:
                return f.read()
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Could not decode {file_path}")

print("Loading and building knowledge bases...")

finance_kb = TextKnowledgeBase(
    name="finance_kb",
    description="Financial documents (news, reports, notes)",
    path=str(FINANCE_DIR),
)

loaded_finance = 0
for file_path in glob.glob(str(FINANCE_DIR / "*.txt")):
    try:
        _ = safe_read_text(file_path)
        finance_kb.load_documents([file_path])
        loaded_finance += 1
        print(f"  ✓ queued: {os.path.basename(file_path)}")
    except Exception as e:
        print(f"  ✗ failed to queue {file_path}: {e}")

print(f"Queued {loaded_finance} text file(s) into finance_kb")

csv_kb = CSVKnowledgeBase(
    name="csv_kb",
    description="CSV financial time series & metrics",
    path=str(CSV_DIR),
)

csv_files = glob.glob(str(CSV_DIR / "*.csv"))
if csv_files:
    try:
        csv_kb.load_documents(csv_files)
        print(f"✓ queued {len(csv_files)} CSV file(s)")
    except Exception as e:
        print(f"✗ failed to queue CSV files: {e}")
else:
    print("No CSV files found; CSV KB empty.")

print("✓ Knowledge bases loaded and indexed (build() not needed in Agno 1.5.9)")

model = Gemini(id="gemini-2.5-flash")

FINANCE_INSTRUCTIONS = [
    "STRICT RAG MODE: You MUST only answer using the provided documents passed in the context.",
    "Do not use any world knowledge beyond the content explicitly provided to you.",
    "If the information is not present in the provided context, reply exactly: 'Not enough information in the provided documents.'",
    "When you use a fact from a document, include a short citation line like: [source: <filename>, start=<index>].",
]

CSV_INSTRUCTIONS = [
    "STRICT RAG MODE: Use ONLY the CSV rows / columns provided in the context.",
    "If a requested date/column/value is missing, reply: 'Data not available in files.'",
    "When summarizing numbers, include the source CSV filename and row indices if possible.",
]

LEADER_INSTRUCTIONS = [
    "You are the Team Leader. Your role is to decompose complex queries into specialized sub-queries.",
    "STEP 1 - ANALYZE: Break down the user query into components that require different expertise:",
    "  - Identify what data/dates are needed (CSV Agent responsibility)",
    "  - Identify what sentiment/analysis/concepts are needed (Finance Agent responsibility)",
    "STEP 2 - DECOMPOSE: Create specialized sub-queries for each agent:",
    "  - For CSV Agent: Ask specifically for data, dates, prices, volumes, metrics",
    "  - For Finance Agent: Ask specifically for sentiment, concepts, analysis, opinions related to the topic",
    "STEP 3 - DELEGATE: Send tailored sub-queries to each agent",
    "STEP 4 - COLLATE: Combine responses from both agents to answer the original query:",
    "  - Show data from CSV Agent",
    "  - Show sentiment/analysis from Finance Agent",
    "  - Connect them to provide a complete answer",
    "  - Present in a clear, structured format",
]

finance_agent = Agent(
    name="Finance_Document_Expert",
    role="Financial Document Specialist",
    model=model,
    knowledge=finance_kb,
    add_context=True,
    instructions=FINANCE_INSTRUCTIONS,
    markdown=True,
    show_tool_calls=True,
)

csv_agent = Agent(
    name="CSV_Data_Analyst",
    role="CSV Financial Data Analyst",
    model=model,
    knowledge=csv_kb,
    add_context=True,
    instructions=CSV_INSTRUCTIONS,
    markdown=True,
    show_tool_calls=True,
)

team_leader = Agent(
    name="Team_Leader",
    role="Multi-Agent Coordinator and Team Leader",
    model=model,
    team=[finance_agent, csv_agent],
    instructions=LEADER_INSTRUCTIONS,
    markdown=True,
    show_tool_calls=True,
)

print("Agents created.")

def retrieve_finance_docs(query: str, top_k: int = TOP_K_TEXT):
    try:
        return finance_kb.search(query, top_k=top_k)
    except:
        return finance_kb.search(query)

def retrieve_csv_rows(query: str, top_k: int = CSV_TOP_K):
    matches = {}
    import re
    date_like = None
    m = re.search(r"(\d{4}[-/]\d{2}[-/]\d{2})", query)
    if m:
        date_like = m.group(1)

    for csv_file in glob.glob(str(CSV_DIR / "*.csv")):
        try:
            df = pd.read_csv(csv_file)
        except:
            df = pd.read_csv(csv_file, engine='python', on_bad_lines='skip')

        matched = pd.DataFrame()
        if date_like:
            for col in df.columns:
                try:
                    parsed = pd.to_datetime(df[col], errors='coerce')
                    mask = parsed.astype(str).str.contains(date_like)
                    if mask.any():
                        matched = df[mask]
                        break
                except:
                    pass
        else:
            kw = query.lower()
            masks = []
            for col in df.columns:
                if df[col].dtype == object:
                    masks.append(df[col].str.lower().str.contains(kw, na=False))
            if masks:
                combined = masks[0]
                for m in masks[1:]:
                    combined = combined | m
                matched = df[combined]

        if not matched.empty:
            matches[os.path.basename(csv_file)] = matched.head(top_k)

    return matches

def build_finance_context(docs):
    parts = []
    for i, d in enumerate(docs):
        txt = getattr(d, 'text', '')
        src = d.metadata.get('source', f'doc_{i}')
        snippet = txt[:800].replace("\n", " ")
        parts.append(f"[source: {src}] {snippet}")
    return "\n\n".join(parts)

def build_csv_context(matches):
    parts = []
    for fname, df in matches.items():
        preview = df.head(10).to_csv(index=False)
        parts.append(f"[CSV: {fname}]\nColumns: {', '.join(df.columns)}\nPreview:\n{preview}")
    return "\n\n".join(parts)

def ask_team(query: str) -> str:
    """
    Main function that decomposes a complex query into sub-queries for specialized agents.
    Each agent gets a tailored query based on their expertise.
    """
    try:
        # STEP 1: Team Leader analyzes and decomposes the query
        decompose_prompt = f"""Analyze this query and decompose it into two specific sub-queries:
        
Original Query: {query}

Provide EXACTLY in this format:
CSV_SUBQUERY: [specific query for CSV data - focus on Date,Open,High,Low,Close,Volume,OpenInt]
FINANCE_SUBQUERY: [specific query for finance documents - focus on sentiment, analysis, concepts]

Example:
Original: "For 2005-03-11 data, can you tell if agreed, neutral or negative"
CSV_SUBQUERY: "What is the stock data for 2005-03-11? Show me the date, open, high, low, close, OpenInt and volume."
FINANCE_SUBQUERY: "What does the document say about agreed, neutral, or negative sentiment? What are the key sentiments or opinions expressed?"
"""
        
        decompose_res = team_leader.run(decompose_prompt)
        decompose_txt = getattr(decompose_res, "content", str(decompose_res))
        
        # Parse the decomposed queries
        csv_subquery = ""
        finance_subquery = ""
        
        for line in decompose_txt.split('\n'):
            if line.startswith('CSV_SUBQUERY:'):
                csv_subquery = line.replace('CSV_SUBQUERY:', '').strip()
            elif line.startswith('FINANCE_SUBQUERY:'):
                finance_subquery = line.replace('FINANCE_SUBQUERY:', '').strip()
        
        print(f"\n📊 CSV Sub-Query: {csv_subquery}")
        print(f"📄 Finance Sub-Query: {finance_subquery}\n")
        
        # STEP 2: Retrieve relevant data for each agent
        finance_hits = retrieve_finance_docs(finance_subquery if finance_subquery else query)
        csv_hits = retrieve_csv_rows(csv_subquery if csv_subquery else query)

        if not finance_hits and not csv_hits:
            return "No information found in knowledge bases."

        finance_context = build_finance_context(finance_hits)
        csv_context = build_csv_context(csv_hits)

        # STEP 3: Send tailored queries to each agent with their specific context
        finance_prompt = (
            "Use ONLY the context provided.\n\nCONTEXT:\n" + finance_context + "\n\nQUERY:\n" + (finance_subquery if finance_subquery else query)
        )
        csv_prompt = (
            "Use ONLY the context provided.\n\nCONTEXT:\n" + csv_context + "\n\nQUERY:\n" + (csv_subquery if csv_subquery else query)
        )

        f_res = finance_agent.run(finance_prompt, context=finance_context)
        c_res = csv_agent.run(csv_prompt, context=csv_context)

        f_txt = getattr(f_res, "content", str(f_res))
        c_txt = getattr(c_res, "content", str(c_res))

        # STEP 4: Team Leader collates responses
        collate_prompt = f"""You are the Team Leader. Collate the responses from both agents to answer the original query.

Original Query: {query}

CSV Agent Response:
{c_txt}

Finance Agent Response:
{f_txt}

Now synthesize these into a complete, coherent answer that combines both data and sentiment/analysis insights."""

        l_res = team_leader.run(collate_prompt)
        l_txt = getattr(l_res, "content", str(l_res))

        return json.dumps({
            "query": query,
            "csv_subquery": csv_subquery,
            "finance_subquery": finance_subquery,
            "csv_agent_response": c_txt,
            "finance_agent_response": f_txt,
            "team_leader_final_answer": l_txt,
        }, indent=2)

    except Exception as e:
        return str(e)

Loading and building knowledge bases...


INFO Loading knowledge base

WARNING  No vector db provided

  ✓ queued: Sentences_50Agree.txt
Queued 1 text file(s) into finance_kb


INFO Loading knowledge base

WARNING  No vector db provided

✓ queued 2 CSV file(s)
✓ Knowledge bases loaded and indexed (build() not needed in Agno 1.5.9)
Agents created.


In [13]:
# Example 1: Basic query without CSV data
query1 = "For 2005-03-11 to 2005-03-28 data, can you tell me if the Operating profit is positive, negative or neutral"



print("Query:", query1)
print("\n" + "="*80 + "\n")
r = ask_team(query1)


Query: For 2005-03-11 to 2005-03-28 data, can you tell me if the Operating profit is positive, negative or neutral



📊 CSV Sub-Query: Retrieve the Operating profit, date, open, high, low, close, OpenInt, and volume for the period 2005-03-11 to 2005-03-28.
📄 Finance Sub-Query: Based on financial analysis, determine if the Operating profit for 2005-03-11 to 2005-03-28 is positive, negative, or neutral.



WARNING  No vector db provided

In [14]:
print(r)

{
  "query": "For 2005-03-11 to 2005-03-28 data, can you tell me if the Operating profit is positive, negative or neutral",
  "csv_subquery": "Retrieve the Operating profit, date, open, high, low, close, OpenInt, and volume for the period 2005-03-11 to 2005-03-28.",
  "finance_subquery": "Based on financial analysis, determine if the Operating profit for 2005-03-11 to 2005-03-28 is positive, negative, or neutral.",
  "csv_agent_response": "Data not available in files for Operating profit.\n\nHere is the requested information from `aap.us.csv` for the period 2005-03-11 to 2005-03-28:\n\n**aap.us.csv (row 1):**\nDate: 2005-03-11\nOpen: 32.994\nHigh: 33.447\nLow: 32.964\nClose: 33.388\nVolume: 1235848\nOpenInt: 0",
  "finance_agent_response": "Not enough information in the provided documents.",
  "team_leader_final_answer": "I understand you were looking for information regarding the operating profit for the period 2005-03-11 to 2005-03-28.\n\nUnfortunately, neither the CSV Agent nor the 